In [1]:
import pandas as pd
import json
import numpy as np

In [3]:
df = pd.read_csv('data/eiti-summary-company-payments.csv')

In [6]:
df = df[df['country'] == "Myanmar"]

In [8]:
df.head()

,created,changed,country,iso3,year,start_date,end_date,company_name,gfs_code,gfs_description,name_of_revenue_stream,currency_code,currency_rate,value_reported,value_reported_as_USD,reporting_url
31969,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1112-E1,"Ordinary taxes on income, profits and capital ...",Revenue Stream: 1112E1 [Ordinary taxes on inco...,MMK,960.5121,4.017128e+10,41822777.0,https://eiti.org/api/v1.0/organisation/31689
31970,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1415-E1,Royalties,Revenue Stream: 1415E1 [Royalties] - Royalties,MMK,960.5121,1.407288e+11,146514378.0,https://eiti.org/api/v1.0/organisation/31689
31971,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1415-E32,Delivered/paid to state-owned enterprise(s),Revenue Stream: 1415E32 [Delivered/paid to sta...,MMK,960.5121,6.139777e+11,639219143.0,https://eiti.org/api/v1.0/organisation/31689
31972,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1415-E32,Delivered/paid to state-owned enterprise(s),Revenue Stream: 1415E32 [Delivered/paid to sta...,MMK,960.5121,1.208082e+11,125774746.0,https://eiti.org/api/v1.0/organisation/31689
31973,2017-02-13T07:00:42+0000,2017-02-13T07:00:42+0000,Myanmar,MMR,2014,"Apr 1, 2013","Mar 31, 2014",Petronas Carigali Myanmar Inc,1415-E32,Delivered/paid to state-owned enterprise(s),Revenue Stream: 1415E32 [Delivered/paid to sta...,MMK,960.5121,4.923598e+09,5126014.0,https://eiti.org/api/v1.0/organisation/31689


## Clean data

In [10]:
df["company_name"].unique()

array(['Petronas Carigali Myanmar Inc', 'TOTAL',
       'Daewoo International Corporation', 'PTT Exploration & Production',
       'Goldpetrol Co Ltd', 'MPRL E&P Pte Ltd', 'Nippon Oil', 'UNOCAL',
       'Chinnery Assets Ltd', 'SNOG Pte Ltd', 'Eternal Mining Company Ltd',
       'Kayah State Mining Production Co. (KMPC)',
       'Myanmar Pongpipat Company Limited', 'Nobel Gold',
       'Tha Byu Mining Co., Ltd.', 'Win Myint Mo Industries Co.,Ltd.',
       'DELCO (Developers Enterepreneurs Liaison Construction Organizers Limited',
       'GPS Joint Venture Co., Ltd.', 'Ngwe Yi Pale Mining Co., Ltd',
       'Max Myanmar Manufacturing Co.,',
       'Triple A Cement International Co., Ltd',
       'Myanmar Wanbao Mining Copper', 'Myanmar Yang Tse Copper Ltd',
       'CNMC Nickel Co Ltd. (MCNICO) (Tagaung Taung Nickel project)',
       'Great Genesis Gems Co., Ltd.', 'MYANMAR IMPERIAL JADE',
       'Richest Gems Co., Ltd', 'LINN LETT WIN YADANAR GEMS',
       'EVER WINNER GEMS', 'AUNG HEIN M

In [13]:
companies_info_df = pd.read_csv('data/eiti_summary_companies_cleaned.csv')

In [14]:
companies_info_df.head()

,Legal name,Identification,Company_name_cl,Identification_cl,Commodities
0,Petronas Carigali Myanmar Inc,NC,PETRONAS CARIGALI MYANMAR INC.(BRANCH OFFICE),16FC/2011-2012,Oil & Gas
1,TOTAL,24FC of 1992-1993,TOTAL E & P MYANMAR (YANGON BRANCH),24FC/1992-1993,Oil & Gas
2,Daewoo International Corporation,1 FC / 2002-2003,Posco Daewoo Corporation (Myanmar E&P Office),1FC/2002-2003,Oil & Gas
3,PTT Exploration & Production,84 FC,PTTEP International Ltd. (Yangon Branch),84FC/1995-1996,Oil & Gas
4,Goldpetrol Co Ltd,173 FC,Goldpetrol Co Ltd,173 FC,Oil & Gas


In [16]:
df = pd.merge(df, companies_info_df, left_on='company_name', right_on='Legal name')

In [23]:
df[df['Commodities'] == 'Gems & Jade']['name_of_revenue_stream'].unique()

array(['Revenue Stream: 1415E1 [Royalties] - Royalties',
       'Revenue Stream: 1415E32 [Delivered/paid to state-owned enterprise(s)] - Sale Split',
       'Revenue Stream: 1415E32 [Delivered/paid to state-owned enterprise(s)] - Emporium Fees / Sale Fees',
       'Revenue Stream: 1422E [Administrative fees for government services] - Supervision Fees',
       'Revenue Stream: 1415E5 [Other rent payments] - Other significant payments (&gt; 50,000 USD)'], dtype=object)

In [24]:
df[df['Commodities'] == 'Gems & Jade']['gfs_description'].unique()

array(['Royalties', 'Delivered/paid to state-owned enterprise(s)',
       'Administrative fees for government services', 'Other rent payments'], dtype=object)

In [33]:
df['revenue_stream_short'] = df['name_of_revenue_stream'].str.split(' - ').str[1]


In [35]:
df_gemsjade = df[df['Commodities'] == 'Gems & jade']

## Remove negative payments for Sankey

In [36]:
df_gemsjade = df_gemsjade[df_gemsjade["value_reported"] >= 0]

## Prepare Source-Target-Value dataframe

In [37]:
links = pd.DataFrame(columns=['source','target','value'])


In [17]:
to_append = df.groupby(['Government Type'],as_index=False)['Value (USD)'].sum()
to_append["target"] = "Total"
to_append.rename(columns = {'Government Type':'target', 'Value (USD)' : 'value'}, inplace = True)
to_append = to_append.sort_values(by=['value'], ascending = False)
links = pd.concat([links,to_append])

print(to_append['value'].sum())
links

2372805274


,source,target,value
0,Total,Federal,1.326789e+09
1,Total,State,9.719232e+08
2,Total,Tribal,6.899835e+07
3,Total,Unknown,5.094728e+06


In [18]:
to_append = df.groupby(['Government Type','Payment Type'],as_index=False)['Value (USD)'].sum()
to_append.rename(columns = {'Government Type':'source','Payment Type':'target', 'Value (USD)' : 'value'}, inplace = True)
to_append = to_append.sort_values(by=['value'], ascending = False)
links = pd.concat([links,to_append])

print(to_append['value'].sum())
links

2372805274


,source,target,value
0,Total,Federal,1.326789e+09
1,Total,State,9.719232e+08
2,Total,Tribal,6.899835e+07
3,Total,Unknown,5.094728e+06
2,Federal,Royalties,7.944091e+08
8,State,Taxes,5.990397e+08
3,Federal,Taxes,4.096844e+08
7,State,Royalties,2.823842e+08
1,Federal,Fees,9.466987e+07
5,State,Fees,8.414347e+07


In [19]:
to_append = df.groupby(['Payment Type','Reporting Company'],as_index=False)['Value (USD)'].sum()
to_append.rename(columns = {'Payment Type':'source','Reporting Company':'target', 'Value (USD)' : 'value'}, inplace = True)
to_append = to_append.sort_values(by=['value'], ascending = False)
links = pd.concat([links,to_append])

print(to_append['value'].sum())
links

2372805274


,source,target,value
0,Total,Federal,1.326789e+09
1,Total,State,9.719232e+08
2,Total,Tribal,6.899835e+07
3,Total,Unknown,5.094728e+06
2,Federal,Royalties,7.944091e+08
8,State,Taxes,5.990397e+08
3,Federal,Taxes,4.096844e+08
7,State,Royalties,2.823842e+08
1,Federal,Fees,9.466987e+07
5,State,Fees,8.414347e+07


In [20]:
unique_list = pd.concat([links['source'], links['target']]).unique()
replace_dict = {k: v for v, k in enumerate(unique_list)}


In [21]:
links_replaced = links.replace({"source": replace_dict,"target": replace_dict})

In [22]:
links_replaced

,source,target,value
0,0,1,1.326789e+09
1,0,2,9.719232e+08
2,0,3,6.899835e+07
3,0,4,5.094728e+06
2,1,5,7.944091e+08
8,2,6,5.990397e+08
3,1,6,4.096844e+08
7,2,5,2.823842e+08
1,1,7,9.466987e+07
5,2,7,8.414347e+07


In [23]:
nodes = pd.DataFrame(unique_list)
nodes.rename(columns = {0:'name'}, inplace = True)

In [24]:
nodes_json= pd.DataFrame(nodes).to_json(orient='records')
nodes_json 

'[{"name":"Total"},{"name":"Federal"},{"name":"State"},{"name":"Tribal"},{"name":"Unknown"},{"name":"Royalties"},{"name":"Taxes"},{"name":"Fees"},{"name":"Production entitlements"},{"name":"Bonuses"},{"name":"Payments for infrastructure improvements"},{"name":"BP P.L.C."},{"name":"ROYAL DUTCH SHELL PLC"},{"name":"Encana Corporation"},{"name":"Barrick Gold Corporation"},{"name":"Agrium Inc."},{"name":"Teck Resources Limited"},{"name":"Rio Tinto PLC"},{"name":"Enerplus Corporation"},{"name":"Kinross Gold Corporation"},{"name":"CNOOC Limited"},{"name":"Potash Corporation of Saskatchewan Inc."},{"name":"Silver Standard Resources Inc."},{"name":"Graymont Limited"},{"name":"Coeur Mining, Inc."},{"name":"Baytex Energy Corp."},{"name":"Total SA"},{"name":"Goldcorp Inc."},{"name":"Crescent Point Energy Corp."},{"name":"Capstone Mining Corp."},{"name":"PetroShale Inc."},{"name":"Energy Fuels Inc"},{"name":"Vermilion Energy Inc."},{"name":"Cameco Corporation"},{"name":"Votorantim Cement North Ame

In [25]:
links_json= pd.DataFrame(links_replaced).to_json(orient='records')
links_json 

'[{"source":0,"target":1,"value":1326789026.0},{"source":0,"target":2,"value":971923173.0},{"source":0,"target":3,"value":68998347.0},{"source":0,"target":4,"value":5094728.0},{"source":1,"target":5,"value":794409147.0},{"source":2,"target":6,"value":599039721.0},{"source":1,"target":6,"value":409684392.0},{"source":2,"target":5,"value":282384235.0},{"source":1,"target":7,"value":94669871.0},{"source":2,"target":7,"value":84143470.0},{"source":3,"target":8,"value":31955077.0},{"source":1,"target":9,"value":28025616.0},{"source":3,"target":5,"value":22103776.0},{"source":3,"target":6,"value":7431555.0},{"source":2,"target":9,"value":6265747.0},{"source":3,"target":7,"value":5229382.0},{"source":4,"target":6,"value":4632565.0},{"source":3,"target":9,"value":2278557.0},{"source":4,"target":7,"value":462163.0},{"source":2,"target":10,"value":90000.0},{"source":5,"target":11,"value":602620019.0},{"source":5,"target":12,"value":374777662.0},{"source":6,"target":13,"value":191680000.0},{"sour

In [26]:
data = { 'links' : json.loads(links_json), 'nodes' : json.loads(nodes_json) }
data_json = json.dumps(data)
data_json = data_json.replace("\\","")
#print(data_json)
#with open('sankey_data.json', 'w') as outfile:
#    json.dump(data_json, outfile)

text_file = open("sankey_data.json", "w")
text_file.write(data_json)
text_file.close()